In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model =  tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/Simulator/kpi_model_v2.h5')
# model.predict()
state = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Simulator/simulator_data_big1.csv',
                    index_col = None)[pos:pos+1].values


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


KeyboardInterrupt: ignored

In [0]:
rand = list(range(1,16)) + list(range(27,37))
k = np.random.binomial(1,.5,len(rand))
k[k==0] = -1
np.random.rand(len(rand)) * k

array([-0.50256019, -0.26214076, -0.71131729,  0.52176441, -0.8584163 ,
       -0.27532678,  0.3613293 , -0.92735347, -0.01833002, -0.89159483,
       -0.8070151 , -0.81634683,  0.6039063 , -0.86674248,  0.79018918,
        0.94796045, -0.28477818,  0.79147042,  0.22354663,  0.20009982,
       -0.88067096,  0.91496748, -0.11099296, -0.0828404 , -0.53750756])

In [0]:
class Simulator(object):
  def __init__(self):
    self.model =  tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/Simulator/kpi_model_v2.h5')
    self.pos = 0
    self.state = np.array(35 * [0])
    self.act = 18 # this is the action column
    self.kpi = 3
    self.stationary = [17,19,20,21,22,23,24,25,26] # these cols won't change during the procedure
    self.rand = list(range(1,16)) + list(range(27,37))# these cols change a random gaussian noise with mean as the beginning status
    self.rand.remove(3)
    self.noise_base =  np.array([0] * 24)
    self.action_space =  np.array([40,50,60,70,80,90,100,110,120,130,140])
    self.cur_kpi = 0
  
  def reset(self):
    self.pos = int(np.round(np.random.rand(1)[0] * 1883614))
    self.state = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Simulator/simulator_data_big1.csv',
                    index_col = None)[self.pos:self.pos+1].values
    # basic possible states from existing points
    self.noise_base = self.state[:,self.rand] * 0.05 # at most 5% change
    self.cur_kpi = self.state[:,self.kpi]
    return self.state[:,list(range(1,3)) + list(range(4,37))]
  
  def step(self,action):
    self.state[:,self.act] = self.action_space[action]
    k = np.random.binomial(1,.5,len(self.rand))
    k[k==0] = -1
    self.state[:,self.rand] = self.noise_base * np.random.rand(len(self.rand)) * k + self.state[:,self.rand]
    pred_kpi = self.model.predict(self.state[:,list(range(1,3)) + list(range(4,37))])
    if self.cur_kpi != 0:
      r = (pred_kpi - self.cur_kpi)/self.cur_kpi
    else:
      r = 0.01
    if r >= 0.05:
      reward = 100
    elif r < -0.05:
      reward = -100
    else:
      reward = 1
    self.cur_kpi = pred_kpi
    
    return self.state[:,list(range(1,3)) + list(range(4,37))],reward
    

In [0]:
"""
Simple policy gradient in Keras
"""
import numpy as np
import pandas as pd

from keras import layers
from keras.models import Model
from keras import backend as K
from keras import utils as np_utils
from keras import optimizers


class Agent(object):

    def __init__(self, input_dim, output_dim, hidden_dims=[32, 32]):
        """The agent for single parameter tuning
        Args:
            input_dim (int): the dimension of state.
                Same as `env.observation_space.shape[0]`
            output_dim (int): the number of discrete actions
                Same as `env.action_space.n`
            hidden_dims (list): hidden dimensions
        Methods:
            private:
                __build_train_fn -> None
                    It creates a train function
                    It's similar to defining `train_op` in Tensorflow
                __build_network -> None
                    It create a base model
                    Its output is each action probability
            public:
                get_action(state) -> action
                fit(state, action, reward) -> None
        """

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.__build_network(input_dim, output_dim, hidden_dims)
        self.__build_train_fn()

    def __build_network(self, input_dim, output_dim, hidden_dims=[32, 32]):
        """Create a base network"""
        self.X = layers.Input(shape=(input_dim,))
        net = self.X

        for h_dim in hidden_dims:
            net = layers.BatchNormalization()(net)
            net = layers.Dense(h_dim)(net)
            net = layers.Activation("relu")(net)

        net = layers.Dense(output_dim)(net)
        net = layers.Activation("softmax")(net)

        self.model = Model(inputs=self.X, outputs=net)
            
         
        #I set output to be 5 at first,(+5,+1,0,-1,-5)

    def __build_train_fn(self):
        """Create a train function
        It replaces `model.fit(X, y)` because we use the output of model and use it for training.
        For example, we need action placeholder
        called `action_one_hot` that stores, which action we took at state `s`.
        Hence, we can update the same action.
        This function will create
        `self.train_fn([state, action_one_hot, discount_reward])`
        which would train the model.
        """
        action_prob_placeholder = self.model.output
        action_onehot_placeholder = K.placeholder(shape=(None, self.output_dim),
                                                  name="action_onehot")
        discount_reward_placeholder = K.placeholder(shape=(None,),
                                                    name="discount_reward")

        action_prob = K.sum(action_prob_placeholder * action_onehot_placeholder, axis=1)
        log_action_prob = K.log(action_prob)

        loss = - log_action_prob * discount_reward_placeholder
        loss = K.mean(loss)

        adam = optimizers.Adam()

        updates = adam.get_updates(params=self.model.trainable_weights,
                                   loss=loss)

        self.train_fn = K.function(inputs=[self.model.input,
                                           action_onehot_placeholder,
                                           discount_reward_placeholder],
                                   outputs=[],
                                   updates=updates)
        
        
    def save(self,path):
      self.model.save(path)
      
    def read_model(self,path):
      self.model = tf.keras.models.load_model(path)

    def get_action(self, state):
        """Returns an action at given `state`
        Args:
            state (1-D or 2-D Array): It can be either 1-D array of shape (state_dimension, )
                or 2-D array shape of (n_samples, state_dimension)
        Returns:
            action: an integer action value ranging from 0 to (n_actions - 1)
        """
        shape = state.shape
        action_prob = np.squeeze(self.model.predict(state))
        return np.random.choice(np.arange(self.output_dim), p=action_prob)

    def fit(self, S, A, R):
        """Train a network
        Args:
            S (2-D Array): `state` array of shape (n_samples, state_dimension)
            A (1-D Array): `action` array of shape (n_samples,)
                It's simply a list of int that stores which actions the agent chose
            R (1-D Array): `reward` array of shape (n_samples,)
                A reward is given after each action.
        """
        action_onehot = np_utils.to_categorical(A, num_classes=self.output_dim)
        discount_reward = compute_discounted_R(R)
        S = S.reshape((-1,35))
        discount_reward = discount_reward.reshape(21)
        assert S.shape[1] == self.input_dim, "{} != {}".format(S.shape[1], self.input_dim)
        assert action_onehot.shape[0] == S.shape[0], "{} != {}".format(action_onehot.shape[0], S.shape[0])
        assert action_onehot.shape[1] == self.output_dim, "{} != {}".format(action_onehot.shape[1], self.output_dim)
        assert len(discount_reward.shape) == 1, "{} != 1".format(len(discount_reward.shape))

        self.train_fn([S, action_onehot, discount_reward])


def compute_discounted_R(R, discount_rate=.99):
    """Returns discounted rewards
    Args:
        R (1-D array): a list of `reward` at each time step
        discount_rate (float): Will discount the future value by this rate
    Returns:
        discounted_r (1-D array): same shape as input `R`
            but the values are discounted
    Examples:
        >>> R = [1, 1, 1]
        >>> compute_discounted_R(R, .99) # before normalization
        [1 + 0.99 + 0.99**2, 1 + 0.99, 1]
    """
    discounted_r = np.zeros_like(R, dtype=np.float32)
    running_add = 0
    for t in reversed(range(len(R))):

        running_add = running_add * discount_rate + R[t]
        discounted_r[t] = running_add

#     discounted_r -= discounted_r.mean() / discounted_r.std()

    return discounted_r


def run_episode(env, agent):
    """Returns an episode reward
    (1) Play until the game is done
    (2) The agent will choose an action according to the policy
    (3) When it's done, it will train from the game play
    Args:
        env (gym.env): Gym environment
        agent (Agent): Game Playing Agent
    Returns:
        total_reward (int): total reward earned during the whole episode
    """
#     done = False
    S = []
    A = []
    R = []

    s = env.reset()

    total_reward = 0

    for i in range(21):
      a = agent.get_action(s)
      S.append(s)
      A.append(a)
      s2, r = env.step(a)
      total_reward += r
        
      R.append(r)

      s = s2

    S = np.array(S)
    A = np.array(A)

    agent.fit(S, A, R)

    return total_reward



output_dim = 11
input_dim = 35
agent = Agent(input_dim, output_dim)
agent.read_model('/content/gdrive/My Drive/Colab Notebooks/Simulator/agent_v2.h5')
m = []
env = Simulator()
for j in range(200):
    r = 0
    for i in range(32):
      reward = run_episode(env, agent)
      r += reward
    agent.save('/content/gdrive/My Drive/Colab Notebooks/Simulator/agent_v2.h5')
    m.append(r/32)
    print(j,m[j])
    
df = pd.DataFrame(m)
df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Simulator/epoch_reward.csv')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
0 41.53125
1 57.5
2 48.28125
3 78.84375
4 41.71875
5 57.3125
6 51.4375
7 23.34375
8 19.6875
9 13.5


In [0]:
import matplotlib.pyplot as plt

plt.plot(m)

NameError: ignored